In [ ]:
from ultralytics import YOLO
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np
import os
import cv2
import glob

In [ ]:
train_path = "categorical-cleared-augmented-clahe"
image_size = 384
batch_size = 16

In [ ]:
model = YOLO('yolov8m-cls.pt')

In [ ]:
model.train(data=train_path, epochs=100, erasing=0, mixup=0.0, project='project', batch=batch_size, imgsz=image_size, patience=10, name=f'{train_path.split("-")[0]}_{batch_size}')

In [ ]:
test_data_path = f"{train_path}/test"
predictions = []
test_images = []
test_classes = []
test_labels =[]
class_folders = os.listdir(test_data_path)
for class_index, class_folder in enumerate(class_folders):
    class_path = os.path.join(test_data_path, class_folder)
    if os.path.isdir(class_path):
        for image_file in glob.glob(os.path.join(class_path, "*.jpg")):
            # Load the image using OpenCV
            image = cv2.imread(image_file)
            # Preprocess the image (resize, normalize, etc.) if needed
            # Perform prediction using your YOLOv8 model
            prediction = model.predict(image, imgsz=384, verbose=False)  # Assuming model.predict() takes an image as input
            for result in prediction:
                predictions.append(result.probs.data.cpu().numpy())
            test_images.append(image)
            test_labels.append(class_folder)
            test_classes.append(class_index)

In [ ]:
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns

predicted_classes = np.argmax(predictions, axis=1)
confusionmatrix = confusion_matrix(test_classes, predicted_classes)
plt.figure(figsize=(4, 4))
sns.set(font_scale=1.2)
sns.heatmap(confusionmatrix, cmap='Blues', annot=True, cbar=True, fmt='g', xticklabels=list(set(test_labels)),
            yticklabels=list(set(test_labels)))

plt.savefig(f'plots/confusion_matrix_{train_path.split("-")[0]}_{batch_size}.jpg')
print(classification_report(test_classes, predicted_classes, digits=4))

In [ ]:
from sklearn.metrics import roc_curve, auc

lb = LabelBinarizer()
lb.fit(test_classes)
test_classes_bin = lb.transform(test_classes)



plt.figure(figsize=(6, 6))
for i in range(len(lb.classes_)):
    # Extract probabilities for class i from predictions
    class_probabilities = [pred[i] for pred in predictions]
    fpr, tpr, _ = roc_curve(test_classes_bin[:, i], class_probabilities)
    roc_auc = roc_auc_score(test_classes_bin[:, i], class_probabilities)
    plt.plot(fpr, tpr, label=f'ROC krivka (plocha = {roc_auc:.2f}) pre triedu {class_folders[i]}')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Falošne pozitívne hodnotenia')
plt.ylabel('Pravdivé pozitívne hodnotenia')
plt.title('Krivka (ROC)')
plt.legend(loc="lower right")
plt.savefig(f'plots/roc_auc_{train_path.split("-")[0]}_{batch_size}.jpg')
plt.show()

In [ ]:
# from sklearn.metrics import roc_curve, auc
# 
# # Initialize an empty list to store probabilities of the positive class
# probabilities_positive_class = []
# 
# # Iterate over each array in the predictions list
# for prediction in predictions:
#     # Extract the probability of the positive class (assuming it's the second element)
#     probability_positive_class = prediction[1]
#     # Append the probability to the list
#     probabilities_positive_class.append(probability_positive_class)
# 
# # Convert the list to a numpy array
# predictions_positive_class = np.array(probabilities_positive_class)
# 
# 
# fpr, tpr, thresholds = roc_curve(test_classes, predictions_positive_class)
# roc_auc = auc(fpr, tpr)
# 
# # Plot ROC curve
# plt.figure(figsize=(6, 6))
# plt.plot(fpr, tpr, label=f'ROC krivka (AUC = {roc_auc:.3f})')
# 
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('Falošne pozitívne hodnotenia')
# plt.ylabel('Pravdivé pozitívne hodnotenia')
# plt.title('Krivka (ROC)')
# plt.legend(loc="lower right")
# plt.savefig(f'plots/roc_auc_{train_path.split("-")[0]}_{batch_size}.jpg')
# plt.show()
